<a href="https://colab.research.google.com/github/ethanrom/NLP-stuff/blob/main/GR_manifesto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

extractive text summarization

In [3]:
# Setup
!pip install -q wordcloud
import wordcloud

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

import pandas as pd
import matplotlib.pyplot as plt
import io
import unicodedata
import numpy as np
import re
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist

def summarize(text, ratio=0.5):
    stop_words = set(stopwords.words("english"))
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]

    fdist = FreqDist(words)
    top_words = [word[0] for word in fdist.most_common(int(len(words) * ratio))]

    sentences = sent_tokenize(text)
    summary = []
    for sentence in sentences:
        for word in top_words:
            if word in sentence.lower():
                summary.append(sentence)
                break

    return " ".join(summary)

if __name__ == '__main__':
    with open("a.txt", "r") as f:
        text = f.read()
    summary = summarize(text)

    with open("output.txt", "w") as f:
        f.write(summary)


counting debt mentions

In [6]:
def count_debt(text):
    words = text.split()
    debt_count = words.count("debt")
    total_count = len(words)
    return (debt_count / total_count) * 100

if __name__ == '__main__':
    with open("a.txt", "r") as f:
        text = f.read()
    debt_percentage = count_debt(text)
    print("The word 'debt' appears in the text {:.2f}% of the time".format(debt_percentage))


The word 'debt' appears in the text 0.03% of the time


Latent Dirichlet Allocation

In [7]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

def preprocess(text):
    return [word for word in simple_preprocess(text) if word not in STOPWORDS]

if __name__ == '__main__':
    with open("a.txt", "r") as f:
        text = f.read()
    documents = text.splitlines()
    processed_docs = [preprocess(doc) for doc in documents]
    dictionary = corpora.Dictionary(processed_docs)
    bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
    lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=2, workers=2)
    for idx, topic in lda_model.print_topics(-1):
        print("Topic: {} \nWords: {}".format(idx, topic))

Topic: 0 
Words: 0.009*"country" + 0.008*"taken" + 0.007*"provided" + 0.006*"government" + 0.006*"sector" + 0.005*"introduced" + 0.005*"people" + 0.004*"steps" + 0.004*"programme" + 0.004*"training"
Topic: 1 
Words: 0.010*"national" + 0.007*"facilities" + 0.006*"ensure" + 0.006*"country" + 0.006*"economic" + 0.005*"state" + 0.005*"people" + 0.005*"security" + 0.005*"programme" + 0.005*"development"
Topic: 2 
Words: 0.011*"government" + 0.007*"people" + 0.007*"national" + 0.007*"public" + 0.006*"country" + 0.006*"services" + 0.005*"billion" + 0.005*"develop" + 0.005*"transport" + 0.004*"agricultural"
Topic: 3 
Words: 0.011*"country" + 0.011*"new" + 0.008*"education" + 0.007*"policy" + 0.006*"ensure" + 0.006*"facilities" + 0.006*"public" + 0.005*"industry" + 0.005*"national" + 0.005*"development"
Topic: 4 
Words: 0.014*"development" + 0.009*"country" + 0.009*"sri" + 0.009*"people" + 0.007*"lanka" + 0.006*"social" + 0.005*"tax" + 0.004*"programme" + 0.004*"international" + 0.004*"economic

extract claim  using Named Entity Recognition

In [9]:
!python -m spacy download en_core_web_sm

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-06 17:43:24.415259: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 52.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
#spaCy model lable CLAIM
!gdown 1Y6A3cpohnPuspB0umy8FEqXHs_2TBHdQ

In [10]:
import spacy

def extract_claims(text):
    # Load the spaCy model
    nlp = spacy.load("en_core_web_sm")
    
    # Parse the text
    doc = nlp(text)
    
    # Iterate over the entities in the document
    claims = []
    for ent in doc.ents:
        # If the entity is a claim, add it to the list of claims
        if ent.label_ == "CLAIM":
            claims.append(ent.text)
    
    return claims

if __name__ == '__main__':
    with open("a.txt", "r") as f:
        text = f.read()
    claims = extract_claims(text)
    for claim in claims:
        print(claim)